<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/sketch/justin/GETSTATS_Clean_NY_2019_Social_Distancing_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clean New York 2019 Social Distancing Dataset

**Instructions:**

1. Mount the google drive to access the data by following **Steps to Mount the Drive**.
2. Provide the drive file path to the data, city name, and year in the fourth code cell. 

**Steps to Mount the Drive:**

1. Execute the second code cell.
2. There will be a link to follow in order to authorize the google account for drive. Go to that link.
3. A code to authorize the google account will be generated. Copy the code generated.
4. Go back to the cell where the process of mounting the drive is running. Paste the generated code from step 3 to the text box in the cell and press enter.

In [1]:
pip install geopandas

     |████████████████████████████████| 1.0MB 6.8MB/s 
     |████████████████████████████████| 15.3MB 253kB/s 
     |████████████████████████████████| 6.5MB 40.2MB/s 


In [2]:
import os
import geopandas
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange

import altair as alt
from vega_datasets import data

# # Allow altair to visualize large datasets
# alt.data_transformers.disable_max_rows()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
city = 'New York'
city_directory = 'ny'
year = '2019'

# Provide a list of CSV file paths to read
path = Path('drive/MyDrive/big-data-project/data/clean-data/')
path_list = [path/city_directory/'social'/year]

In [5]:
#Appending into dataframe

def get_df(path_list):
    df = []
    first = True
    for directory in tqdm(path_list, desc='Years'):
        f_list = os.listdir(directory)
        csv_files = [directory/f for f in f_list if f.endswith('.csv')]
        for csv in tqdm(csv_files, desc='Days'):
            if first:
                df = pd.read_csv(csv, dtype={'cbg': object})
                first = False
            else:
                df = df.append(pd.read_csv(csv, dtype={'cbg': object}), ignore_index=True)
    return df.sort_values(by=['date_range_start'])

In [6]:
%%time

df = get_df(path_list)



CPU times: user 9min 25s, sys: 11.7 s, total: 9min 37s
Wall time: 14min 47s


In [7]:
df

,Unnamed: 0,cbg,date_range_start,date_range_end,device_count,distance_traveled_from_home,bucketed_distance_traveled,median_dwell_at_bucketed_distance_traveled,completely_home_device_count,median_home_dwell_time,bucketed_home_dwell_time,at_home_by_each_hour,part_time_work_behavior_devices,full_time_work_behavior_devices,destination_cbgs,delivery_behavior_devices,median_non_home_dwell_time,candidate_device_count,bucketed_away_from_home_time,median_percentage_time_home,bucketed_percentage_time_home,mean_home_dwell_time,mean_non_home_dwell_time,mean_distance_traveled_from_home
0,146,360470064002,2019-01-01T00:00:00-05:00,2019-01-02T00:00:00-05:00,60,620,"{""16001-50000"":7,""0"":25,"">50000"":1,""2001-8000""...","{""16001-50000"":77,"">50000"":1086,""<1000"":38,""20...",29,795,"{""721-1080"":6,""361-720"":9,""61-360"":13,""<60"":8,...","[32,32,31,32,30,31,29,32,30,29,30,31,27,31,30,...",4,3,"{""360810846021"":1,""360470138001"":1,""3604700660...",4,16,159,"{""21-45"":6,""481-540"":2,""721-840"":1,""301-360"":2...",98,"{""0-25"":4,""76-100"":41,""51-75"":5,""26-50"":5}",772,174,2320
4184,147266,360470432001,2019-01-01T00:00:00-05:00,2019-01-02T00:00:00-05:00,69,215,"{""16001-50000"":2,""0"":37,"">50000"":1,""2001-8000""...","{""16001-50000"":20,"">50000"":90,""<1000"":63,""2001...",36,1270,"{""721-1080"":11,""361-720"":2,""61-360"":6,""<60"":11...","[46,45,48,49,49,51,48,43,44,47,46,45,44,45,41,...",4,2,"{""360610039002"":1,""360470430003"":1,""3604702980...",2,3,158,"{""21-45"":4,""481-540"":4,""541-600"":1,""46-60"":1,""...",100,"{""0-25"":11,""76-100"":46,""51-75"":9,""26-50"":1}",939,134,383
4183,147265,360470296002,2019-01-01T00:00:00-05:00,2019-01-02T00:00:00-05:00,114,340,"{""16001-50000"":7,""0"":45,"">50000"":5,""2001-8000""...","{""16001-50000"":98,"">50000"":78,""<1000"":51,""2001...",49,891,"{""721-1080"":28,""361-720"":18,""61-360"":14,""<60"":...","[66,65,69,69,73,73,73,73,69,66,68,61,59,56,54,...",6,4,"{""360470298002"":1,""360470558001"":3,""3608503190...",2,42,280,"{""21-45"":3,""481-540"":2,""541-600"":5,""46-60"":4,""...",95,"{""0-25"":13,""76-100"":67,""51-75"":14,""26-50"":9}",782,212,3424
4182,147094,360810846012,2019-01-01T00:00:00-05:00,2019-01-02T00:00:00-05:00,84,4161,"{""16001-50000"":5,""0"":43,"">50000"":6,""2001-8000""...","{""16001-50000"":36,"">50000"":624,""<1000"":150,""20...",43,740,"{""721-1080"":5,""361-720"":15,""61-360"":5,""<60"":21...","[37,42,45,44,48,50,46,53,51,53,48,45,45,44,40,...",1,1,"{""360810086001"":1,""360470129013"":1,""3605940540...",1,0,240,"{""21-45"":1,""541-600"":4,""46-60"":4,""721-840"":1,""...",100,"{""0-25"":16,""76-100"":60,""51-75"":4,""26-50"":2}",740,151,83602
4181,147093,360810838004,2019-01-01T00:00:00-05:00,2019-01-02T00:00:00-05:00,76,943,"{""16001-50000"":3,""0"":42,"">50000"":3,""2001-8000""...","{""16001-50000"":55,"">50000"":156,""<1000"":44,""200...",38,913,"{""721-1080"":11,""361-720"":9,""61-360"":16,""<60"":3...","[31,38,41,41,45,45,46,44,49,48,47,39,45,46,43,...",3,2,"{""360610020001"":2,""360810164002"":1,""3606101310...",3,6,183,"{""21-45"":8,""481-540"":1,""541-600"":1,""721-840"":3...",100,"{""0-25"":9,""76-100"":61,""51-75"":4,""26-50"":1}",815,143,9459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2286062,76078,360470570004,2019-12-31T00:00:00-05:00,2020-01-01T00:00:00-05:00,70,1565,"{""16001-50000"":11,""0"":20,"">50000"":1,""2001-8000...","{""16001-50000"":85,"">50000"":76,""<1000"":121,""200...",19,820,"{""721-1080"":19,""361-720"":16,""61-360"":7,""<60"":6...","[52,56,51,51,51,53,49,46,35,30,31,29,27,23,29,...",11,3,"{""360470039001"":1,""360610088001"":1,""3606101310...",7,210,131,"{""21-45"":1,""481-540"":3,""541-600"":1,""1201-1320""...",78,"{""0-25"":6,""76-100"":34,""51-75"":21,""26-50"":5}",804,267,6926
2286061,76075,360050245022,2019-12-31T00:00:00-05:00,2020-01-01T00:00:00-05:00,26,1657,"{""16001-50000"":8,""0"":4,""2001-8000"":6,""1-1000"":...","{""16001-50000"":96,""<1000"":123,""2

In [8]:
# Selecting the columsn that we need for the analysis
# col = ['date_range_start',
#        'cbg',
#        'device_count',  # Total active devices
#        'completely_home_device_count',
#        'part_time_work_behavior_devices',
#        'full_time_work_behavior_devices',
#        'delivery_behavior_devices',
#        'median_percentage_time_home',
#        'median_home_dwell_time',
#        'mean_home_dwell_time',
#        'median_non_home_dwell_time',
#        'mean_non_home_dwell_time',
#        'distance_traveled_from_home', # Median Distance traveled
#        'mean_distance_traveled_from_home']
col = ['date_range_start',
       'cbg',
       'device_count',  # Total active devices
       'completely_home_device_count',
       'part_time_work_behavior_devices',
       'full_time_work_behavior_devices']
devices_mobility = df[col]

In [9]:
#Renaming columns
devices_mobility = devices_mobility.rename(columns={'date_range_start':'date','device_count':'total', 'completely_home_device_count':'completely_home','part_time_work_behavior_devices':'part_time_work', 'full_time_work_behavior_devices':'full_time_work'})
devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work
0,2019-01-01T00:00:00-05:00,360470064002,60,29,4,3
4184,2019-01-01T00:00:00-05:00,360470432001,69,36,4,2
4183,2019-01-01T00:00:00-05:00,360470296002,114,49,6,4
4182,2019-01-01T00:00:00-05:00,360810846012,84,43,1,1
4181,2019-01-01T00:00:00-05:00,360810838004,76,38,3,2
...,...,...,...,...,...,...
2286062,2019-12-31T00:00:00-05:00,360470570004,70,19,11,3
2286061,2019-12-31T00:00:00-05:00,360050245022,26,6,1,1
2286060,2019-12-31T00:00:00-05:00,360050075004,31,14,1,1
2286058,2019-12-31T00:00:00-05:00,360810616022,39,3,1,1


In [10]:
# devices_mobility

In [11]:
#Listing out the device count columns

device_columns = ['completely_home','part_time_work','full_time_work']

In [12]:
#Calculating percentage for all the device columns

for column in device_columns :
    devices_mobility['percentage_'+column]= devices_mobility.apply(lambda row: row[column]/row['total']*100.0 if row['total']!=0.0 else 0.0, axis=1)
devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work
0,2019-01-01T00:00:00-05:00,360470064002,60,29,4,3,48.333333,6.666667,5.000000
4184,2019-01-01T00:00:00-05:00,360470432001,69,36,4,2,52.173913,5.797101,2.898551
4183,2019-01-01T00:00:00-05:00,360470296002,114,49,6,4,42.982456,5.263158,3.508772
4182,2019-01-01T00:00:00-05:00,360810846012,84,43,1,1,51.190476,1.190476,1.190476
4181,2019-01-01T00:00:00-05:00,360810838004,76,38,3,2,50.000000,3.947368,2.631579
...,...,...,...,...,...,...,...,...,...
2286062,2019-12-31T00:00:00-05:00,360470570004,70,19,11,3,27.142857,15.714286,4.285714
2286061,2019-12-31T00:00:00-05:00,360050245022,26,6,1,1,23.076923,3.846154,3.846154
2286060,2019-12-31T00:00:00-05:00,360050075004,31,14,1,1,45.161290,3.225806,3.225806
2286058,2019-12-31T00:00:00-05:00,360810616022,39,3,1,1,7.692308,2.564103,2.564103


In [13]:
#Normalising the percentage columns

for column in device_columns:
        devices_mobility['norm_'+column] = (devices_mobility['percentage_'+ column] - devices_mobility['percentage_'+column].mean()) / devices_mobility['percentage_'+column].std()

devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
0,2019-01-01T00:00:00-05:00,360470064002,60,29,4,3,48.333333,6.666667,5.000000,1.353290,-0.298193,-0.135040
4184,2019-01-01T00:00:00-05:00,360470432001,69,36,4,2,52.173913,5.797101,2.898551,1.720819,-0.468676,-0.602902
4183,2019-01-01T00:00:00-05:00,360470296002,114,49,6,4,42.982456,5.263158,3.508772,0.841233,-0.573358,-0.467044
4182,2019-01-01T00:00:00-05:00,360810846012,84,43,1,1,51.190476,1.190476,1.190476,1.626708,-1.371830,-0.983183
4181,2019-01-01T00:00:00-05:00,360810838004,76,38,3,2,50.000000,3.947368,2.631579,1.512784,-0.831326,-0.662340
...,...,...,...,...,...,...,...,...,...,...,...,...
2286062,2019-12-31T00:00:00-05:00,360470570004,70,19,11,3,27.142857,15.714286,4.285714,-0.674553,1.475642,-0.294067
2286061,2019-12-31T00:00:00-05:00,360050245022,26,6,1,1,23.076923,3.846154,3.846154,-1.063647,-0.851170,-0.391930
2286060,2019-12-31T00:00:00-05:00,360050075004,31,14,1,1,45.161290,3.225806,3.225806,1.049739,-0.972792,-0.530043
2286058,2019-12-31T00:00:00-05:00,360810616022,39,3,1,1,7.692308,2.564103,2.564103,-2.535893,-1.102523,-0.677363


In [14]:
#Conversion to datetime format

devices_mobility['date'] = pd.to_datetime(devices_mobility['date'],utc= True)
devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
0,2019-01-01 05:00:00+00:00,360470064002,60,29,4,3,48.333333,6.666667,5.000000,1.353290,-0.298193,-0.135040
4184,2019-01-01 05:00:00+00:00,360470432001,69,36,4,2,52.173913,5.797101,2.898551,1.720819,-0.468676,-0.602902
4183,2019-01-01 05:00:00+00:00,360470296002,114,49,6,4,42.982456,5.263158,3.508772,0.841233,-0.573358,-0.467044
4182,2019-01-01 05:00:00+00:00,360810846012,84,43,1,1,51.190476,1.190476,1.190476,1.626708,-1.371830,-0.983183
4181,2019-01-01 05:00:00+00:00,360810838004,76,38,3,2,50.000000,3.947368,2.631579,1.512784,-0.831326,-0.662340
...,...,...,...,...,...,...,...,...,...,...,...,...
2286062,2019-12-31 05:00:00+00:00,360470570004,70,19,11,3,27.142857,15.714286,4.285714,-0.674553,1.475642,-0.294067
2286061,2019-12-31 05:00:00+00:00,360050245022,26,6,1,1,23.076923,3.846154,3.846154,-1.063647,-0.851170,-0.391930
2286060,2019-12-31 05:00:00+00:00,360050075004,31,14,1,1,45.161290,3.225806,3.225806,1.049739,-0.972792,-0.530043
2286058,2019-12-31 05:00:00+00:00,360810616022,39,3,1,1,7.692308,2.564103,2.564103,-2.535893,-1.102523,-0.677363


In [15]:
# Normalize time

devices_mobility['date'] = devices_mobility['date'].dt.normalize()
devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
0,2019-01-01 00:00:00+00:00,360470064002,60,29,4,3,48.333333,6.666667,5.000000,1.353290,-0.298193,-0.135040
4184,2019-01-01 00:00:00+00:00,360470432001,69,36,4,2,52.173913,5.797101,2.898551,1.720819,-0.468676,-0.602902
4183,2019-01-01 00:00:00+00:00,360470296002,114,49,6,4,42.982456,5.263158,3.508772,0.841233,-0.573358,-0.467044
4182,2019-01-01 00:00:00+00:00,360810846012,84,43,1,1,51.190476,1.190476,1.190476,1.626708,-1.371830,-0.983183
4181,2019-01-01 00:00:00+00:00,360810838004,76,38,3,2,50.000000,3.947368,2.631579,1.512784,-0.831326,-0.662340
...,...,...,...,...,...,...,...,...,...,...,...,...
2286062,2019-12-31 00:00:00+00:00,360470570004,70,19,11,3,27.142857,15.714286,4.285714,-0.674553,1.475642,-0.294067
2286061,2019-12-31 00:00:00+00:00,360050245022,26,6,1,1,23.076923,3.846154,3.846154,-1.063647,-0.851170,-0.391930
2286060,2019-12-31 00:00:00+00:00,360050075004,31,14,1,1,45.161290,3.225806,3.225806,1.049739,-0.972792,-0.530043
2286058,2019-12-31 00:00:00+00:00,360810616022,39,3,1,1,7.692308,2.564103,2.564103,-2.535893,-1.102523,-0.677363


In [16]:
# Check the number of rows for each CBG

print('Min: {}\nMax: {}'.format(min(set(devices_mobility.groupby('cbg').count()['date'])), max(set(devices_mobility.groupby('cbg').count()['date']))))

Min: 1
Max: 365


In [17]:
for col in devices_mobility.columns:
    print('\n')
    print(col)
#     if devices_mobility[col].dtype
#     print(devices_mobility[col].dtype)
    print('Min: {}\nMax: {}'.format(devices_mobility[col].min(), devices_mobility[col].max()))



date
Min: 2019-01-01 00:00:00+00:00
Max: 2019-12-31 00:00:00+00:00


cbg
Min: 360050001000
Max: 360859901000


total
Min: 5
Max: 17185


completely_home
Min: 1
Max: 10466


part_time_work
Min: 1
Max: 653


full_time_work
Min: 1
Max: 438


percentage_completely_home
Min: 0.3574620196604111
Max: 91.82509505703422


percentage_part_time_work
Min: 0.16891891891891891
Max: 66.66666666666666


percentage_full_time_work
Min: 0.0999000999000999
Max: 56.25


norm_completely_home
Min: -3.237808124120669
Max: 5.5152777053642215


norm_part_time_work
Min: -1.572111523866646
Max: 11.46513490817982


norm_full_time_work
Min: -1.2259865758719086
Max: 11.275132935699647


In [18]:
# Print all Null and NaN values

print('We have {} null values.'.format(len(devices_mobility[devices_mobility.isna().any(axis=1)])))

We have 0 null values.


In [19]:
# Grouping based on day

grouped_dm = devices_mobility.groupby(devices_mobility['date']).mean()
grouped_dm

,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
date,,,,,,,,,,
2019-01-01 00:00:00+00:00,87.196554,40.884174,2.631940,1.589343,45.625279,3.445967,2.397087,1.094140,-0.929628,-0.714547
2019-01-02 00:00:00+00:00,87.486685,35.184660,6.510126,5.042258,38.763633,7.804475,6.263688,0.437508,-0.075119,0.146304
2019-01-03 00:00:00+00:00,86.287036,32.974645,7.073194,5.346197,36.682705,8.551397,6.638771,0.238372,0.071319,0.229812
2019-01-04 00:00:00+00:00,87.155098,32.594862,7.619754,5.396043,35.796867,9.205950,6.667589,0.153601,0.199648,0.236228
2019-01-05 00:00:00+00:00,87.451891,40.827669,3.633956,1.850008,45.222852,4.445734,2.582264,1.055630,-0.733619,-0.673319
...,...,...,...,...,...,...,...,...,...,...
2019-12-27 00:00:00+00:00,65.613999,20.573182,4.274235,2.712851,30.353171,6.706018,4.402689,-0.367339,-0.290478,-0.268024
2019-12-28 00:00:00+00:00,65.488921,22.191296,3.108879,1.763271,33.019258,5.164828,3.323233,-0.112205,-0.592636,-0.508352
2019-12-29 00:00:00+00:00,67.006537,24.505102,2.946588,1.540976,35.349221,4.704489,2.835890,0.110763,-0.682888,-0.616853


In [20]:
grouped_dm.reset_index()

,date,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
0,2019-01-01 00:00:00+00:00,87.196554,40.884174,2.631940,1.589343,45.625279,3.445967,2.397087,1.094140,-0.929628,-0.714547
1,2019-01-02 00:00:00+00:00,87.486685,35.184660,6.510126,5.042258,38.763633,7.804475,6.263688,0.437508,-0.075119,0.146304
2,2019-01-03 00:00:00+00:00,86.287036,32.974645,7.073194,5.346197,36.682705,8.551397,6.638771,0.238372,0.071319,0.229812
3,2019-01-04 00:00:00+00:00,87.155098,32.594862,7.619754,5.396043,35.796867,9.205950,6.667589,0.153601,0.199648,0.236228
4,2019-01-05 00:00:00+00:00,87.451891,40.827669,3.633956,1.850008,45.222852,4.445734,2.582264,1.055630,-0.733619,-0.673319
...,...,...,...,...,...,...,...,...,...,...,...
360,2019-12-27 00:00:00+00:00,65.613999,20.573182,4.274235,2.712851,30.353171,6.706018,4.402689,-0.367339,-0.290478,-0.268024
361,2019-12-28 00:00:00+00:00,65.488921,22.191296,3.108879,1.763271,33.019258,5.164828,3.323233,-0.112205,-0.592636,-0.508352
362,2019-12-29 00:00:00+00:00,67.006537,24.505102,2.946588,1.540976,35.349221,4.704489,2.835890,0.110763,-0.682888,-0.616853
363,2019-12-30 00:00:00+00:00,65.658218,21.890324,3.793241,2.725650,32.051070,5.878297,4.372094,-0.204857,-0.452757,-0.274836


In [21]:
#Visualising mobility

title = '{} {}'.format(city,year)
alt.Chart(grouped_dm.reset_index()).mark_line().encode(
    x=alt.X('date', title='Date'),
    y=alt.Y(alt.repeat('row'), type='quantitative')
).properties(
    width=300,
    height=250
).repeat(
    row=['percentage_completely_home','percentage_part_time_work','percentage_full_time_work']
).interactive()

alt.RepeatChart(...)

![ny-daily-2019](https://github.com/chouhandiksha/bigdataproject/raw/main/media/social-dist/ny-daily-2019.png)

In [22]:
#Adding month column for further usage

devices_mobility['month'] = devices_mobility['date'].dt.month
devices_mobility

,date,cbg,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work,month
0,2019-01-01 00:00:00+00:00,360470064002,60,29,4,3,48.333333,6.666667,5.000000,1.353290,-0.298193,-0.135040,1
4184,2019-01-01 00:00:00+00:00,360470432001,69,36,4,2,52.173913,5.797101,2.898551,1.720819,-0.468676,-0.602902,1
4183,2019-01-01 00:00:00+00:00,360470296002,114,49,6,4,42.982456,5.263158,3.508772,0.841233,-0.573358,-0.467044,1
4182,2019-01-01 00:00:00+00:00,360810846012,84,43,1,1,51.190476,1.190476,1.190476,1.626708,-1.371830,-0.983183,1
4181,2019-01-01 00:00:00+00:00,360810838004,76,38,3,2,50.000000,3.947368,2.631579,1.512784,-0.831326,-0.662340,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2286062,2019-12-31 00:00:00+00:00,360470570004,70,19,11,3,27.142857,15.714286,4.285714,-0.674553,1.475642,-0.294067,12
2286061,2019-12-31 00:00:00+00:00,360050245022,26,6,1,1,23.076923,3.846154,3.846154,-1.063647,-0.851170,-0.391930,12
2286060,2019-12-31 00:00:00+00:00,360050075004,31,14,1,1,45.161290,3.225806,3.225806,1.049739,-0.972792,-0.530043,12
2286058,2019-12-31 00:00:00+00:00,360810616022,39,3,1,1,7.692308,2.564103,2.564103,-2.535893,-1.102523,-0.677363,12


In [23]:
#Grouping data on basis of month value

grouped_dm = devices_mobility.groupby([devices_mobility['month']]).mean()
grouped_dm

,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work
month,,,,,,,,,,
1,90.991158,38.096721,5.951565,4.341536,40.228179,6.912426,5.272107,0.577660,-0.250010,-0.074459
2,89.622259,35.977141,6.820775,4.282661,38.795410,8.017269,5.346345,0.440549,-0.033400,-0.057931
3,84.711106,31.523672,7.095637,4.192132,36.108085,8.789636,5.526480,0.183383,0.118027,-0.017826
4,84.389308,30.355290,7.168533,4.169180,35.055139,8.731856,5.366273,0.082621,0.106699,-0.053494
5,83.595717,28.945013,7.238603,4.508303,33.897067,8.846064,5.753708,-0.028202,0.129090,0.032763
6,80.559397,27.921774,6.146871,3.827314,34.093693,7.823738,5.063704,-0.009386,-0.071343,-0.120857
7,74.604653,25.594096,5.585426,3.221515,33.769410,7.775292,4.736156,-0.040419,-0.080841,-0.193782
8,77.789317,26.130069,5.310551,3.117349,32.432593,7.387532,4.637040,-0.168347,-0.156863,-0.215849
9,92.438930,30.586317,7.000818,4.669175,31.530800,8.242584,5.811663,-0.254645,0.010775,0.045666


In [24]:


d = []
for col in ['percentage_completely_home','percentage_part_time_work','percentage_full_time_work']:
        for m in range(1,13):
            d.append({'month': m, 'column': col, 'value': grouped_dm[col][m]})
d = alt.Data(values=d)

In [25]:
#Visualizing percentage of mobility of each mobility type.

title = '{} {}'.format(city,year)
alt.Chart(d, title=title).mark_line().encode(
    x=alt.X('month:O', title='Month'),
    y=alt.Y('value:Q', type='quantitative', title='Percentage'),
    color='column:N'
).properties(
    width=300,
    height=250
).interactive()

alt.Chart(...)

![ny-monthly-2019](https://github.com/chouhandiksha/bigdataproject/raw/main/media/social-dist/ny-monthly-2019.png)

**Get Stats**

In [26]:
df_summary = devices_mobility.describe()
df_summary

,total,completely_home,part_time_work,full_time_work,percentage_completely_home,percentage_part_time_work,percentage_full_time_work,norm_completely_home,norm_part_time_work,norm_full_time_work,month
count,2.290248e+06,2.290248e+06,2.290248e+06,2.290248e+06,2.290248e+06,2.290248e+06,2.290248e+06,2.290248e+06,2.290248e+06,2.290248e+06,2.290248e+06
mean,8.034527e+01,2.843472e+01,6.304288e+00,4.110137e+00,3.419177e+01,8.187627e+00,5.606548e+00,3.141520e-13,4.354980e-13,-2.230468e-13,6.524348e+00
std,1.000178e+02,4.663538e+01,6.281790e+00,4.238384e+00,1.044976e+01,5.100598e+00,4.491606e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.448154e+00
min,5.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.574620e-01,1.689189e-01,9.990010e-02,-3.237808e+00,-1.572112e+00,-1.225987e+00,1.000000e+00
25%,4.800000e+01,1.400000e+01,2.000000e+00,1.000000e+00,2.727273e+01,4.166667e+00,2.127660e+00,-6.621251e-01,-7.883313e-01,-7.745312e-01,4.000000e+00
50%,6.700000e+01,2.300000e+01,5.000000e+00,3.000000e+00,3.428571e+01,7.407407e+00,4.255319e+00,8.989615e-03,-1.529663e-01,-3.008343e-01,7.000000e+00
75%,9.400000e+01,3.500000e+01,8.000000e+00,6.000000e+00,4.122807e+01,1.111111e+01,7.865169e+00,6.733452e-01,5.731650e-01,5.028536e-01,1.000000e+01
max,1.718500e+04,1.046600e+04,6.530000e+02,4.380000e+02,9.182510e+01,6.666667e+01,5.625000e+01,5.515278e+00,1.146513e+01,1.127513e+01,1.200000e+01


In [27]:
filename = 'drive/MyDrive/big-data-project/sketch/justin/{}_social_summary_{}.csv'.format(city_directory, year)
df_summary.to_csv(filename)